In [67]:
import pandas as pd
import numpy as np

data = pd.read_csv('puzzle_batches/small_batch.csv')
data = data[['FEN','Moves','Rating','RatingDeviation','Popularity','NbPlays','Themes']].dropna()

In [68]:
unique_themes = set(' '.join([row for row in data['Themes']]).split())
for theme in unique_themes:
    data[theme] = 0

In [69]:
for row_index, row_themes in data['Themes'].iteritems():
    row_themes_list = row_themes.split(' ')
    for theme in row_themes_list:
        data[theme][row_index] = 1

/home/bastosgabriel/miniconda3/envs/chessenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [70]:
data.drop('Themes', inplace=True, axis=1)

In [71]:
data.corr().abs()[['endgame']]

,endgame
Rating,0.197243
RatingDeviation,0.659055
Popularity,0.529009
NbPlays,0.207550
short,0.577350
long,0.333333
mate,0.333333
defensiveMove,0.149071
mateIn2,0.333333
hangingPiece,0.654654
